In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1109997,2021-03-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1109998,2021-03-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1109999,2021-03-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1110000,2021-03-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
29903,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29905,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29907,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29909,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29911,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
685546,2021-03-22,Arkansas,Arkansas,5001,2024,35.00,5000,Arkansas,AR,Arkansas,State,3017804
685548,2021-03-23,Arkansas,Arkansas,5001,2027,35.00,5000,Arkansas,AR,Arkansas,State,3017804
685550,2021-03-24,Arkansas,Arkansas,5001,2031,35.00,5000,Arkansas,AR,Arkansas,State,3017804
685552,2021-03-25,Arkansas,Arkansas,5001,2068,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1109997,2021-03-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1109998,2021-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1109999,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1110000,2021-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
430,2021-03-26,Snohomish,Washington,32426,558.00,53061,WA,County,822083,3944.37,67.88
858,2021-03-26,Cook,Illinois,492837,10177.00,17031,IL,County,5150233,9569.22,197.60
1285,2021-03-26,Orange,California,265536,4684.00,6059,CA,County,3175692,8361.52,147.50
1711,2021-03-26,Maricopa,Arizona,522624,9630.00,4013,AZ,County,4485414,11651.63,214.70
2137,2021-03-26,Los Angeles,California,1217686,23055.00,6037,CA,County,10039107,12129.43,229.65
...,...,...,...,...,...,...,...,...,...,...,...
1109465,2021-03-26,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1109630,2021-03-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1109764,2021-03-26,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1109894,2021-03-26,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
430,2021-03-26,Snohomish,Washington,32426,558.00,53061,WA,County,822083,3944.37,67.88,67.88,3944.37
858,2021-03-26,Cook,Illinois,492837,10177.00,17031,IL,County,5150233,9569.22,197.60,197.60,9569.22
1285,2021-03-26,Orange,California,265536,4684.00,6059,CA,County,3175692,8361.52,147.50,147.50,8361.52
1711,2021-03-26,Maricopa,Arizona,522624,9630.00,4013,AZ,County,4485414,11651.63,214.70,214.70,11651.63
2137,2021-03-26,Los Angeles,California,1217686,23055.00,6037,CA,County,10039107,12129.43,229.65,229.65,12129.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109465,2021-03-26,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1109630,2021-03-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1109764,2021-03-26,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1109894,2021-03-26,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
716769,2021-03-26,Hale,Alabama,2158,73.00,1065,AL,County,14651,14729.37,498.26,498.26,14729.37,1
780039,2021-03-26,Clarke,Alabama,3460,58.00,1025,AL,County,23622,14647.36,245.53,245.53,14647.36,2
561081,2021-03-26,Lowndes,Alabama,1358,53.00,1085,AL,County,9726,13962.57,544.93,544.93,13962.57,3
417666,2021-03-26,Franklin,Alabama,4183,82.00,1059,AL,County,31362,13337.80,261.46,261.46,13337.80,4
497596,2021-03-26,Etowah,Alabama,13631,342.00,1055,AL,County,102268,13328.70,334.42,334.42,13328.70,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684101,2021-03-26,Converse,Wyoming,982,17.00,56009,WY,County,13822,7104.62,122.99,122.99,7104.62,19
684465,2021-03-26,Sublette,Wyoming,693,7.00,56035,WY,County,9831,7049.13,71.20,71.20,7049.13,20
850046,2021-03-26,Lincoln,Wyoming,1328,12.00,56023,WY,County,19830,6696.92,60.51,60.51,6696.92,21
945712,2021-03-26,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
561081,2021-03-26,Lowndes,Alabama,1358,53.00,1085,AL,County,9726,13962.57,544.93,544.93,13962.57,3,1
716769,2021-03-26,Hale,Alabama,2158,73.00,1065,AL,County,14651,14729.37,498.26,498.26,14729.37,1,2
238935,2021-03-26,Walker,Alabama,7032,274.00,1127,AL,County,63521,11070.35,431.35,431.35,11070.35,28,3
604998,2021-03-26,Greene,Alabama,903,34.00,1063,AL,County,8111,11133.03,419.18,419.18,11133.03,26,4
560349,2021-03-26,Crenshaw,Alabama,1495,57.00,1041,AL,County,13772,10855.36,413.88,413.88,10855.36,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603537,2021-03-26,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
850046,2021-03-26,Lincoln,Wyoming,1328,12.00,56023,WY,County,19830,6696.92,60.51,60.51,6696.92,21,20
828924,2021-03-26,Uinta,Wyoming,2119,12.00,56041,WY,County,20226,10476.61,59.33,59.33,10476.61,4,21
237816,2021-03-26,Teton,Wyoming,3562,9.00,56039,WY,County,23464,15180.70,38.36,38.36,15180.70,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107386,2021-03-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1107387,2021-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1107388,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1107389,2021-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
713797,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,1.00
713798,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,0.00
713799,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,0.00
713800,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,0.00
713801,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14729.37,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864210,2021-03-22,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
864211,2021-03-23,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
864212,2021-03-24,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
864213,2021-03-25,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
558833,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,1.00,0.00
558834,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
558835,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
558836,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
558837,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556998,2021-03-22,Albany,Wyoming,3947,11.00,56001,WY,County,38880,10151.75,28.29,28.29,10187.76,23,7,11.00,0.00
556999,2021-03-23,Albany,Wyoming,3951,11.00,56001,WY,County,38880,10162.04,28.29,28.29,10187.76,23,7,4.00,0.00
557000,2021-03-24,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10187.76,23,7,1.00,0.00
557001,2021-03-25,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10187.76,23,7,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
285155,2021-03-26,Imperial,California,27352,711.00,6025,CA,County,181215,15093.67,392.35,392.35,15093.67,1,2,27.00,0.00
2137,2021-03-26,Los Angeles,California,1217686,23055.00,6037,CA,County,10039107,12129.43,229.65,229.65,12129.43,2,5,782.00,35.00
499016,2021-03-26,Inyo,California,1369,35.00,6027,CA,County,18039,7589.11,194.02,194.02,7589.11,3,24,1.00,0.00
142127,2021-03-26,San Bernardino,California,290315,4222.00,6071,CA,County,2180085,13316.68,193.66,193.66,13316.68,4,4,160.00,52.00
63621,2021-03-26,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,94.00,4.00
78858,2021-03-26,Tulare,California,48960,811.00,6107,CA,County,466195,10502.04,173.96,173.96,10502.04,6,10,29.00,3.00
29517,2021-03-26,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,124.00,34.00
50635,2021-03-26,San Joaquin,California,69448,1250.00,6077,CA,County,762148,9112.14,164.01,164.01,9112.14,8,19,153.00,6.00
28747,2021-03-26,Fresno,California,98602,1593.00,6019,CA,County,999101,9869.07,159.44,159.44,9869.07,9,15,165.00,8.00
604940,2021-03-26,Kings,California,22578,241.00,6031,CA,County,152940,14762.65,157.58,157.58,14762.65,10,3,17.00,2.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1053520,2021-03-26,Lassen,California,5649,1.00,6035,CA,County,30573,18477.09,3.27,3.27,18477.09,56,1,12.00,0.00
285155,2021-03-26,Imperial,California,27352,711.00,6025,CA,County,181215,15093.67,392.35,392.35,15093.67,1,2,27.00,0.00
604940,2021-03-26,Kings,California,22578,241.00,6031,CA,County,152940,14762.65,157.58,157.58,14762.65,10,3,17.00,2.00
142127,2021-03-26,San Bernardino,California,290315,4222.00,6071,CA,County,2180085,13316.68,193.66,193.66,13316.68,4,4,160.00,52.00
2137,2021-03-26,Los Angeles,California,1217686,23055.00,6037,CA,County,10039107,12129.43,229.65,229.65,12129.43,2,5,782.00,35.00
29517,2021-03-26,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,124.00,34.00
178641,2021-03-26,Kern,California,105912,1206.00,6029,CA,County,900202,11765.36,133.97,133.97,11765.36,14,7,44.00,8.00
452297,2021-03-26,Merced,California,30419,434.00,6047,CA,County,277680,10954.70,156.30,156.30,10954.70,11,8,47.00,2.00
63621,2021-03-26,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,94.00,4.00
78858,2021-03-26,Tulare,California,48960,811.00,6107,CA,County,466195,10502.04,173.96,173.96,10502.04,6,10,29.00,3.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
285155,2021-03-26,Imperial,California,27352,711.00,6025,CA,County,181215,15093.67,392.35,392.35,15093.67,1,2,27.00,0.00
2137,2021-03-26,Los Angeles,California,1217686,23055.00,6037,CA,County,10039107,12129.43,229.65,229.65,12129.43,2,5,782.00,35.00
499016,2021-03-26,Inyo,California,1369,35.00,6027,CA,County,18039,7589.11,194.02,194.02,7589.11,3,24,1.00,0.00
142127,2021-03-26,San Bernardino,California,290315,4222.00,6071,CA,County,2180085,13316.68,193.66,193.66,13316.68,4,4,160.00,52.00
63621,2021-03-26,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,94.00,4.00
78858,2021-03-26,Tulare,California,48960,811.00,6107,CA,County,466195,10502.04,173.96,173.96,10502.04,6,10,29.00,3.00
29517,2021-03-26,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,124.00,34.00
50635,2021-03-26,San Joaquin,California,69448,1250.00,6077,CA,County,762148,9112.14,164.01,164.01,9112.14,8,19,153.00,6.00
28747,2021-03-26,Fresno,California,98602,1593.00,6019,CA,County,999101,9869.07,159.44,159.44,9869.07,9,15,165.00,8.00
604940,2021-03-26,Kings,California,22578,241.00,6031,CA,County,152940,14762.65,157.58,157.58,14762.65,10,3,17.00,2.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15093.67,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3732,03/22/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,3.00,0.00
3733,03/23/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3734,03/24/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3735,03/25/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15093.67,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15093.67,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3815,03/22/21,Kings,22524,238.00,14727.34,155.62,157.58,14762.65,10,3,13.00,0.00
3816,03/23/21,Kings,22532,239.00,14732.57,156.27,157.58,14762.65,10,3,8.00,1.00
3817,03/24/21,Kings,22546,239.00,14741.73,156.27,157.58,14762.65,10,3,14.00,0.00
3818,03/25/21,Kings,22561,239.00,14751.54,156.27,157.58,14762.65,10,3,15.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)